# data-scientist-exercise01

Primary References:
- https://www.datacamp.com/community/tutorials/decision-tree-classification-python
- http://queirozf.com/entries/one-hot-encoding-a-feature-on-a-pandas-dataframe-an-example
- https://www.geeksforgeeks.org/how-to-get-column-names-in-pandas-dataframe/
- https://www.geeksforgeeks.org/confusion-matrix-machine-learning/ 
- StackOverflow
- StackExchange
- ect.

### 4. Import the "flattened" table (or CSV file) into your open source analytic environment of choice (R, Python, Java, etc.) and stage it for analysis.

In [1]:
import csv
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import matplotlib.pyplot as plt
from sklearn.feature_selection import SelectFromModel
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from sklearn.model_selection import train_test_split # Import train_test_split function
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.metrics import confusion_matrix 
from sklearn.metrics import accuracy_score 
from sklearn.metrics import classification_report 
from sklearn.tree import export_graphviz
from sklearn.externals.six import StringIO  
from IPython.display import Image  
import sys
!{sys.executable} -m pip install pydotplus
import sys
!{sys.executable} -m pip install graphviz
from sklearn.naive_bayes import GaussianNB
import warnings
warnings.filterwarnings('ignore') 

/srv/conda/envs/notebook/lib/python3.7/site-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)


In [2]:
mydata= "flattenedRecords.csv"
df = pd.read_csv(mydata) 
df.head()
df.columns
#df.isnull().sum()

Index(['id', 'age', 'workClass', 'eduLevel', 'eduNum', 'maritalStatus',
       'occupation', 'relationship', 'race', 'sex', 'capitalGain',
       'capitalLoss', 'hoursPerWeek', 'country', 'over50K'],
      dtype='object')

### 6. Split the data into training, validation, and test data sets.

In [3]:
#what is the current # of 0 and 1 in column
(df['over50K'].value_counts()/df['over50K'].count())*100

0    76.071823
1    23.928177
Name: over50K, dtype: float64

In [4]:
# converting all '?' to NaN
df = df.mask(df=='?', float('NaN'))

#SKLearn only takes # so I have to change all of the Strings to #
# use pd.concat to join the new columns with your original dataframe
df = pd.concat([df,pd.get_dummies(df['workClass'], prefix='workClass')],axis=1)
df = pd.concat([df,pd.get_dummies(df['eduLevel'], prefix='eduLevel')],axis=1)
df = pd.concat([df,pd.get_dummies(df['maritalStatus'], prefix='maritalStatus')],axis=1)
df = pd.concat([df,pd.get_dummies(df['occupation'], prefix='occupation')],axis=1)
df = pd.concat([df,pd.get_dummies(df['relationship'], prefix='relationship')],axis=1)
df = pd.concat([df,pd.get_dummies(df['race'], prefix='race')],axis=1)
df = pd.concat([df,pd.get_dummies(df['sex'], prefix='sex')],axis=1)
df = pd.concat([df,pd.get_dummies(df['country'], prefix='country')],axis=1)
                     
# get rid of the original column
df.drop(['workClass'],axis=1, inplace=True)
df.drop(['eduLevel'],axis=1, inplace=True)
df.drop(['maritalStatus'],axis=1, inplace=True)
df.drop(['occupation'],axis=1, inplace=True)
df.drop(['relationship'],axis=1, inplace=True)
df.drop(['race'],axis=1, inplace=True)
df.drop(['sex'],axis=1, inplace=True)
df.drop(['country'],axis=1, inplace=True)

#show all column names
for col in df.columns: 
    print(col) 

id
age
eduNum
capitalGain
capitalLoss
hoursPerWeek
over50K
workClass_Federal-gov
workClass_Local-gov
workClass_Never-worked
workClass_Private
workClass_Self-emp-inc
workClass_Self-emp-not-inc
workClass_State-gov
workClass_Without-pay
eduLevel_10th
eduLevel_11th
eduLevel_12th
eduLevel_1st-4th
eduLevel_5th-6th
eduLevel_7th-8th
eduLevel_9th
eduLevel_Assoc-acdm
eduLevel_Assoc-voc
eduLevel_Bachelors
eduLevel_Doctorate
eduLevel_HS-grad
eduLevel_Masters
eduLevel_Preschool
eduLevel_Prof-school
eduLevel_Some-college
maritalStatus_Divorced
maritalStatus_Married-AF-spouse
maritalStatus_Married-civ-spouse
maritalStatus_Married-spouse-absent
maritalStatus_Never-married
maritalStatus_Separated
maritalStatus_Widowed
occupation_Adm-clerical
occupation_Armed-Forces
occupation_Craft-repair
occupation_Exec-managerial
occupation_Farming-fishing
occupation_Handlers-cleaners
occupation_Machine-op-inspct
occupation_Other-service
occupation_Priv-house-serv
occupation_Prof-specialty
occupation_Protective-serv


In [5]:
#split dataset in target and other
target = ['over50K']

other = ['id', 'age', 'eduNum', 'capitalGain', 'capitalLoss', 'hoursPerWeek',
         'workClass_Federal-gov', 'workClass_Local-gov',
       'workClass_Never-worked', 'workClass_Private', 'workClass_Self-emp-inc',
       'workClass_Self-emp-not-inc', 'workClass_State-gov',
       'workClass_Without-pay', 'eduLevel_10th', 'eduLevel_11th',
       'eduLevel_12th', 'eduLevel_1st-4th', 'eduLevel_5th-6th',
       'eduLevel_7th-8th', 'eduLevel_9th', 'eduLevel_Assoc-acdm',
       'eduLevel_Assoc-voc', 'eduLevel_Bachelors', 'eduLevel_Doctorate',
       'eduLevel_HS-grad', 'eduLevel_Masters', 'eduLevel_Preschool',
       'eduLevel_Prof-school', 'eduLevel_Some-college',
       'maritalStatus_Divorced', 'maritalStatus_Married-AF-spouse',
       'maritalStatus_Married-civ-spouse',
       'maritalStatus_Married-spouse-absent', 'maritalStatus_Never-married',
       'maritalStatus_Separated', 'maritalStatus_Widowed',
       'occupation_Adm-clerical', 'occupation_Armed-Forces',
       'occupation_Craft-repair', 'occupation_Exec-managerial',
       'occupation_Farming-fishing', 'occupation_Handlers-cleaners',
       'occupation_Machine-op-inspct', 'occupation_Other-service',
       'occupation_Priv-house-serv', 'occupation_Prof-specialty',
       'occupation_Protective-serv', 'occupation_Sales',
       'occupation_Tech-support', 'occupation_Transport-moving',
       'relationship_Husband', 'relationship_Not-in-family',
       'relationship_Other-relative', 'relationship_Own-child',
       'relationship_Unmarried', 'relationship_Wife',
       'race_Amer-Indian-Eskimo', 'race_Asian-Pac-Islander', 'race_Black',
       'race_Other', 'race_White', 'sex_Female', 'sex_Male','country_Cambodia', 'country_Canada','country_China',
         'country_Columbia','country_Cuba','country_Dominican-Republic','country_Ecuador','country_El-Salvador',
         'country_England','country_France','country_Germany','country_Greece','country_Guatemala','country_Haiti',
         'country_Holand-Netherlands','country_Honduras','country_Hong','country_Hungary','country_India','country_Iran',
         'country_Ireland','country_Italy','country_Jamaica','country_Japan','country_Laos','country_Mexico','country_Nicaragua',
         'country_Outlying-US(Guam-USVI-etc)','country_Peru','country_Philippines','country_Poland','country_Portugal','country_Puerto-Rico', 'country_Scotland',
       'country_South', 'country_Taiwan', 'country_Thailand',
       'country_Trinadad&Tobago', 'country_United-States', 'country_Vietnam',
       'country_Yugoslavia']

X = df[other]
y = df[target] 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=1) # 50% training and 20% test
X_train, X_validation, y_train, y_validation = train_test_split(X_train, y_train, test_size = 0.2, random_state=2)# 50% training and 20% Validation

#CHECK We expect the training features number of columns to match the testing feature number of columns and the number of rows to match for the respective training and testing features and the labels
print('X_train Shape of XTrain:', X_train.shape)
print('y_train Shape of YTrain:', y_train.shape)
print('X_test Shape of XTest:', X_test.shape)
print('y_test Shape of YTest:', y_test.shape)
print('Shape of XValidation:', X_validation.shape)
print('Shape of YValidation:', y_validation.shape)

X_train Shape of XTrain: (31258, 105)
y_train Shape of YTrain: (31258, 1)
X_test Shape of XTest: (9769, 105)
y_test Shape of YTest: (9769, 1)
Shape of XValidation: (7815, 105)
Shape of YValidation: (7815, 1)


### 7. Develop a model that predicts whether individuals, based on the census variables provided, make over $50,000/year. Use over_50k as the target variable.

In [6]:
# Decision Tree classifer object
clf = DecisionTreeClassifier()

# Train Decision Tree Classifer
clf = clf.fit(X_train,y_train)

#Predict the response for test dataset
y_pred = clf.predict(X_test)

#ValueError: could not convert string to float: '?' 
    #for this reason I will now convert
    
results = confusion_matrix(y_test, y_pred) 
print('Confusion Matrix :')
print(results,"\n") 
print("Accuracy:", metrics.accuracy_score(y_test, y_pred),"\n")
print('Report : ')
print(classification_report(y_test, y_pred))

Confusion Matrix :
[[6421 1009]
 [ 860 1479]] 

Accuracy: 0.8086805200122837 

Report : 
              precision    recall  f1-score   support

           0       0.88      0.86      0.87      7430
           1       0.59      0.63      0.61      2339

    accuracy                           0.81      9769
   macro avg       0.74      0.75      0.74      9769
weighted avg       0.81      0.81      0.81      9769



In [ ]:
# BETTER Decision Tree classifer object
clf = DecisionTreeClassifier(criterion="entropy", max_depth=12)

# Train Decision Tree Classifer
clf = clf.fit(X_train,y_train)

#Predict the response for test dataset
y_pred = clf.predict(X_test)

results = confusion_matrix(y_test, y_pred) 
print('Confusion Matrix :')
print(results,"\n") 
print("Accuracy:", metrics.accuracy_score(y_test, y_pred),"\n")
print('Report : ')
print(classification_report(y_test, y_pred))

In [ ]:
#print(dict(zip(X_test.columns, clf.feature_importances_)))

In [ ]:
for name, importance in zip(df[other], clf.feature_importances_):
    print(name, "=", importance)

importances = clf.feature_importances_
indices = np.argsort(importances)
plt.title('Feature Importances')
plt.barh(range(len(indices)), importances[indices], color='b', align='center')
plt.yticks(range(len(indices)), [other[i] for i in indices])
plt.xlabel('Relative Importance')
plt.show()

In [ ]:
# Random Forest Classifier
clf = RandomForestClassifier(n_estimators=100)

#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)

results = confusion_matrix(y_test, y_pred) 
print('Confusion Matrix :')
print(results,"\n") 
print("Accuracy:", metrics.accuracy_score(y_test, y_pred),"\n")
print('Report : ')
print(classification_report(y_test, y_pred))

In [ ]:
for name, importance in zip(df[other], clf.feature_importances_):
    print(name, "=", importance)

importances = clf.feature_importances_
indices = np.argsort(importances)
plt.title('Feature Importances')
plt.barh(range(len(indices)), importances[indices], color='b', align='center')
plt.yticks(range(len(indices)), [other[i] for i in indices])
plt.xlabel('Relative Importance')
plt.show()

In [ ]:
#apply SelectKBest class to extract top 10 best features
bestfeatures = SelectKBest(score_func=chi2, k=10)
fit = bestfeatures.fit(X_train,y_train)
dfscores = pd.DataFrame(fit.scores_)
dfcolumns = pd.DataFrame(X.columns)
#concat two dataframes for better visualization 
featureScores = pd.concat([dfcolumns,dfscores],axis=1)
featureScores.columns = ['Specs','Score']  #naming the dataframe columns
print(featureScores.nlargest(10,'Score'))  #print 10 best features

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
import matplotlib.pyplot as plt
model = ExtraTreesClassifier()
model.fit(X_train,y_train)
print(model.feature_importances_) #use inbuilt class feature_importances of tree based classifiers
#plot graph of feature importances for better visualization
feat_importances = pd.Series(model.feature_importances_, index=X.columns)
feat_importances.nlargest(10).plot(kind='barh')
plt.show()